### Q1 - Embedding The Query

In [1]:
%pip install qdrant-client[fastembed]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from fastembed import TextEmbedding

query = 'I just discovered the course. Can I join now?'

embedding_model = TextEmbedding(model_name='jinaai/jina-embeddings-v2-small-en')
query_vector = next(embedding_model.embed(query))

c:\Users\usuario\Programming\llm-zoomcamp\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
type(query_vector)

numpy.ndarray

Getting the minimum value present.

In [4]:
min(query_vector)

np.float64(-0.11726373885183883)

The embedding is already normalized, which is why its length/magnitude is 1.

In [5]:
import numpy as np

np.linalg.norm(query_vector)

np.float64(1.0)

`cosine_similarity(a, b) = (a · b) / (|a| * |b|)`. Since `|a| = 1` and `|b| = 1`, we can calculate the cosine similarity by just performing the dot product.

In [6]:
query_vector.dot(query_vector)

np.float64(1.0000000000000002)

The vector has an agle of 0 and degrees with itself thus a cosine value of 1. It is the most similar.

### Q2 - Cosine Similarity With Another Vector

In [7]:
doc = 'Can I still join the course after the start date?'
doc_vector = next(embedding_model.embed(doc))

np.dot(query_vector, doc_vector)

np.float64(0.9008528895674548)

### Q3 - Ranking By Cosine

In [8]:
docs = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
    'section': 'General course-related questions',
    'question': 'Course - Can I still join the course after the start date?',
    'course': 'data-engineering-zoomcamp'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
    'section': 'General course-related questions',
    'question': 'Course - Can I follow the course after it finishes?',
    'course': 'data-engineering-zoomcamp'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?',
    'course': 'data-engineering-zoomcamp'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
    'section': 'General course-related questions',
    'question': 'Course - What can I do before the course starts?',
    'course': 'data-engineering-zoomcamp'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
    'section': 'General course-related questions',
    'question': 'How can we contribute to the course?',
    'course': 'data-engineering-zoomcamp'}
]

In [9]:
vectors = list(
    embedding_model.embed(
        documents=[doc['text'] for doc in docs]
    )
)

**Solution 1**: Computing vector by vector.

In [10]:
index, max_cosine = None, -1
for i, vector in enumerate(vectors):
    cosine = np.dot(query_vector, vector)
    if cosine > max_cosine:
        index, max_cosine = i, cosine

print(f"Doc index: {index}")
print(f"Cosine similarity: {max_cosine}")

Doc index: 1
Cosine similarity: 0.8182378150042889


In [11]:
docs[index]

{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 'section': 'General course-related questions',
 'question': 'Course - Can I follow the course after it finishes?',
 'course': 'data-engineering-zoomcamp'}

**Solution 2**: Computing with matrix.

In [12]:
matrix = np.array(vectors)
cosines = matrix.dot(query_vector)

index = np.argmax(cosines)
max_cosine = cosines[index]

print(f"Doc index: {index}")
print(f"Cosine similarity: {max_cosine}")

Doc index: 1
Cosine similarity: 0.8182378150042889


In [13]:
docs[index]

{'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 'section': 'General course-related questions',
 'question': 'Course - Can I follow the course after it finishes?',
 'course': 'data-engineering-zoomcamp'}

### Q4 - Ranking By Cosine (V2)

In [14]:
vectors = list(
    embedding_model.embed(
        documents=[
            doc['question'] + ' ' + doc['text'] 
            for doc in docs
        ]
    )
)

In [15]:
matrix = np.array(vectors)
cosines = matrix.dot(query_vector)

index = np.argmax(cosines)
max_cosine = cosines[index]

print(f"Doc index: {index}")
print(f"Cosine similarity: {max_cosine}")

Doc index: 0
Cosine similarity: 0.8514543236908068


In [16]:
docs[index]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

Now the highest ranked document contains the actual answer to the query. This is because we also added the question to the embedding, which contributes semantically.

### Q5 - Selecting The Embedding Model

In [24]:
supported_models = TextEmbedding.list_supported_models()

model_info = min(supported_models, key=lambda d: d['dim'])
model_info

{'model': 'BAAI/bge-small-en',
 'sources': {'hf': 'Qdrant/bge-small-en',
  'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz',
  '_deprecated_tar_struct': True},
 'model_file': 'model_optimized.onnx',
 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.',
 'license': 'mit',
 'size_in_GB': 0.13,
 'additional_files': [],
 'dim': 384,
 'tasks': {}}

In [25]:
model_handle = model_info['model']

### Q6 - Indexing With Qdrant

In [26]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [27]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")

In [28]:
collection_name = 'rag'
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=client.get_embedding_size(model_handle),
        distance=models.Distance.COSINE
    )
)

True

In [29]:
client.upload_collection(
    collection_name=collection_name,
    vectors=[
        models.Document(
            text=doc['question'] + ' ' + doc['text'], 
            model=model_handle
        ) 
        for doc in documents
    ],
    payload=[doc for doc in documents],
    ids=[index for index in range(0, len(documents))],
)

In [30]:
search_result = client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query, 
        model=model_handle
    ),
    limit=1
)
search_result.points[0]

ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)

The result is different as we are using a smaller embedding model.